In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#### Functions

In [2]:
def dist(stop_lat, stop_lon,next_o_lat,next_o_lon):
    return np.arccos(
        np.sin(np.radians(stop_lat)) * np.sin(np.radians(next_o_lat)) + 
        np.cos(np.radians(stop_lat)) * np.cos(np.radians(next_o_lat)) * 
            np.cos(np.radians(stop_lon) - np.radians(next_o_lon))
    ) * 6371

#### Read Origin/Next-Origin Pairs

In [3]:
trips_origins = pd.read_csv('/local/tarciso/masters/data/bus_trips/test/enhanced-buste/2017_05_09_user_trips.csv', parse_dates=['o_boarding_datetime','o_gps_datetime','next_o_boarding_datetime','next_o_gps_datetime'])

In [4]:
len(trips_origins)

177756

In [5]:
trips_origins.sort_values(['cardNum','o_boarding_id']).head(20)

,cardNum,o_boarding_id,o_boarding_datetime,o_route,o_busCode,o_tripNum,o_stopPointId,o_gps_datetime,o_stop_lat,o_stop_lon,next_o_boarding_id,next_o_boarding_datetime,next_o_route,next_o_busCode,next_o_tripNum,next_o_stopPointId,next_o_gps_datetime,next_o_stop_lat,next_o_stop_lon,boardings_timediff
32608,233641.0,2,2017-05-09 06:39:21,000,01026,NaN,41863,NaT,-25.423639,-49.202354,3,2017-05-09 12:08:17,021,04020,NaN,14508,NaT,-25.441614,-49.346740,0 days 05:28:56.000000000
32609,233641.0,3,2017-05-09 12:08:17,021,04020,NaN,14508,NaT,-25.441614,-49.346740,2,2017-05-09 06:39:21,000,01026,NaN,41863,NaT,-25.423639,-49.202354,NaN
93114,257342.0,4,2017-05-09 07:48:58,511,EA172,4.0,31195,2017-05-09 07:47:51,-25.498750,-49.247840,5,2017-05-09 17:24:51,000,01021,NaN,26037,NaT,-25.471734,-49.237478,0 days 09:35:53.000000000
93115,257342.0,5,2017-05-09 17:24:51,000,01021,NaN,26037,NaT,-25.471734,-49.237478,4,2017-05-09 07:48:58,511,EA172,4.0,31195,2017-05-09 07:47:51,-25.498750,-49.247840,NaN
134081,304127.0,9,2017-05-09 10:24:50,175,BC010,5.0,31748,2017-05-09 10:22:28,-25.450968,-49.254072,10,2017-05-09 10:57:36,370,LC016,3.0,3377,2017-05-09 10:42:18,-25.428142,-49.246865,0 days 00:32:46.000000000
134082,304127.0,10,2017-05-09 10:57:36,370,LC016,3.0,3377,2017-05-09 10:42:18,-25.428142,-49.246865,9,2017-05-09 10:24:50,175,BC010,5.0,31748,2017-05-09 10:22:28,-25.450968,-49.254072,NaN
165949,304627.0,12,2017-05-09 14:17:23,370,BC032,6.0,3377,2017-05-09 14:16:09,-25.428142,-49.246865,13,2017-05-09 16:24:40,370,LC020,6.0,30303,2017-05-09 15:45:37,-25.455659,-49.323150,0 days 02:07:17.000000000
165950,304627.0,13,2017-05-09 16:24:40,370,LC020,6.0,30303,2017-05-09 15:45:37,-25.455659,-49.323150,12,2017-05-09 14:17:23,370,BC032,6.0,3377,2017-05-09 14:16:09,-25.428142,-49.246865,NaN
138994,305601.0,14,2017-05-09 11:06:44,000,05007,NaN,14517,NaT,-25.377432,-49.262446,15,2017-05-09 13:32:37,000,00045,NaN,43723,NaT,-25.433715,-49.270258,0 days 02:25:53.000000000
138995,305601.0,15,2017-05-09 13:32:37,000,00045,NaN,43723,NaT,-25.433715,-49.270258,14,2017-05-09 11:06:44,000,05007,NaN,14517,NaT,-25.377432,-49.262446,NaN


In [6]:
trips_origins['dist_between_origins'] = trips_origins.apply(lambda x: dist(x['o_stop_lat'],x['o_stop_lon'],x['next_o_stop_lat'],x['next_o_stop_lon']), axis=1)
trips_origins['boardings_timediff'] = pd.to_timedelta(trips_origins['boardings_timediff'])

/local/tarciso/programs/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in arccos
  """


In [7]:
trips_origins['boardings_timediff'] = np.where(np.isnat(trips_origins['boardings_timediff']),
                                             trips_origins['o_boarding_datetime'] - trips_origins['next_o_boarding_datetime'],
                                             trips_origins['boardings_timediff'])

In [8]:
trips_origins.sort_values(['cardNum','o_boarding_id']).head(50)

,cardNum,o_boarding_id,o_boarding_datetime,o_route,o_busCode,o_tripNum,o_stopPointId,o_gps_datetime,o_stop_lat,o_stop_lon,...,next_o_boarding_datetime,next_o_route,next_o_busCode,next_o_tripNum,next_o_stopPointId,next_o_gps_datetime,next_o_stop_lat,next_o_stop_lon,boardings_timediff,dist_between_origins
32608,233641.0,2,2017-05-09 06:39:21,000,01026,NaN,41863,NaT,-25.423639,-49.202354,...,2017-05-09 12:08:17,021,04020,NaN,14508,NaT,-25.441614,-49.346740,05:28:56,14.636225
32609,233641.0,3,2017-05-09 12:08:17,021,04020,NaN,14508,NaT,-25.441614,-49.346740,...,2017-05-09 06:39:21,000,01026,NaN,41863,NaT,-25.423639,-49.202354,05:28:56,14.636225
93114,257342.0,4,2017-05-09 07:48:58,511,EA172,4.0,31195,2017-05-09 07:47:51,-25.498750,-49.247840,...,2017-05-09 17:24:51,000,01021,NaN,26037,NaT,-25.471734,-49.237478,09:35:53,3.178960
93115,257342.0,5,2017-05-09 17:24:51,000,01021,NaN,26037,NaT,-25.471734,-49.237478,...,2017-05-09 07:48:58,511,EA172,4.0,31195,2017-05-09 07:47:51,-25.498750,-49.247840,09:35:53,3.178960
134081,304127.0,9,2017-05-09 10:24:50,175,BC010,5.0,31748,2017-05-09 10:22:28,-25.450968,-49.254072,...,2017-05-09 10:57:36,370,LC016,3.0,3377,2017-05-09 10:42:18,-25.428142,-49.246865,00:32:46,2.639323
134082,304127.0,10,2017-05-09 10:57:36,370,LC016,3.0,3377,2017-05-09 10:42:18,-25.428142,-49.246865,...,2017-05-09 10:24:50,175,BC010,5.0,31748,2017-05-09 10:22:28,-25.450968,-49.254072,00:32:46,2.639323
165949,304627.0,12,2017-05-09 14:17:23,370,BC032,6.0,3377,2017-05-09 14:16:09,-25.428142,-49.246865,...,2017-05-09 16:24:40,370,LC020,6.0,30303,2017-05-09 15:45:37,-25.455659,-49.323150,02:07:17,8.248431
165950,304627.0,13,2017-05-09 16:24:40,370,LC020,6.0,30303,2017-05-09 15:45:37,-25.455659,-49.323150,...,2017-05-09 14:17:23,370,BC032,6.0,3377,2017-05-09 14:16:09,-25.428142,-49.246865,02:07:17,8.248431
138994,305601.0,14,2017-05-09 11:06:44,000,05007,NaN,14517,NaT,-25.377432,-49.262446,...,2017-05-09 13:32:37,000,00045,NaN,43723,NaT,-25.433715,-49.270258,02:25:53,6.307331
138995,305601.0,15,2017-05-09 13:32:37,000,00045,NaN,43723,NaT,-25.433715,-49.270258,...,2017-05-09 11:06:44,000,05007,NaN,14517,NaT,-25.377432,-49.262446,02:25:53,6.307331


In [9]:
trips_origins.dist_between_origins.describe()

count    176588.000000
mean          5.467415
std           4.161731
min           0.000000
25%           2.052165
50%           5.142366
75%           8.200742
max          28.419669
Name: dist_between_origins, dtype: float64

In [10]:
trips_origins.boardings_timediff.describe()

count                    177756
mean     0 days 05:49:04.257307
std      0 days 04:05:58.037363
min             0 days 00:00:01
25%      0 days 01:38:57.750000
50%             0 days 05:49:27
75%             0 days 09:49:12
max             0 days 16:26:35
Name: boardings_timediff, dtype: object

In [11]:
len(trips_origins)

177756

In [12]:
trips_origins_clean = trips_origins[(trips_origins['dist_between_origins'] > 1.5) &
                                    (trips_origins['boardings_timediff'] > pd.Timedelta('5 min'))]

In [13]:
len(trips_origins_clean)

139498

In [14]:
trips_origins.dtypes

cardNum                             float64
o_boarding_id                         int64
o_boarding_datetime          datetime64[ns]
o_route                              object
o_busCode                            object
o_tripNum                           float64
o_stopPointId                         int64
o_gps_datetime               datetime64[ns]
o_stop_lat                          float64
o_stop_lon                          float64
next_o_boarding_id                    int64
next_o_boarding_datetime     datetime64[ns]
next_o_route                         object
next_o_busCode                       object
next_o_tripNum                      float64
next_o_stopPointId                    int64
next_o_gps_datetime          datetime64[ns]
next_o_stop_lat                     float64
next_o_stop_lon                     float64
boardings_timediff          timedelta64[ns]
dist_between_origins                float64
dtype: object

#### Read OTP Suggestions

In [15]:
otp_suggestions = pd.read_csv('/local/tarciso/masters/data/bus_trips/test/otp-itineraries/2017_05_09_otp_itineraries.csv', parse_dates=['date','otp_start_time','otp_end_time'])
otp_suggestions['otp_start_time'] = otp_suggestions['otp_start_time'] - pd.Timedelta('10800 s')
otp_suggestions['otp_end_time'] = otp_suggestions['otp_end_time'] - pd.Timedelta('10800 s')
otp_suggestions['route'] = otp_suggestions['route'].astype(str)
otp_suggestions['route'] = np.where(otp_suggestions['mode'] == 'BUS',
                                    otp_suggestions['route'].astype(str).str.replace("\.0",'').str.zfill(3),
                                    otp_suggestions['route'])

In [16]:
len(otp_suggestions.drop_duplicates(subset=['user_trip_id']))

956

In [17]:
otp_suggestions.head()

,date,user_trip_id,itinerary_id,leg_id,otp_start_time,otp_end_time,mode,route,from_stop_id,to_stop_id,otp_duration_mins
0,2017-05-09,1496,1,1,2017-05-09 05:14:11,2017-05-09 05:17:49,WALK,nan,NaN,NaN,3.633333
1,2017-05-09,1496,1,2,2017-05-09 05:17:50,2017-05-09 05:45:00,BUS,272,27222.0,26365.0,27.166667
2,2017-05-09,1496,1,3,2017-05-09 05:45:00,2017-05-09 05:47:44,WALK,nan,NaN,NaN,2.733333
3,2017-05-09,1496,1,4,2017-05-09 05:55:00,2017-05-09 06:15:56,BUS,304,25891.0,26286.0,20.933333
4,2017-05-09,1496,1,5,2017-05-09 06:15:56,2017-05-09 06:16:35,WALK,nan,NaN,NaN,0.650000


In [18]:
len(otp_suggestions)

43688

#### Adding Parent Stop data to OTP Suggestions

In [19]:
stops_df = pd.read_csv('/local/tarciso/data/gtfs/curitiba/stops.txt')

In [20]:
stops_df.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_timezone,wheelchair_boarding
0,70,104505,Terminal Campina do Siqueira - 303 - Centenári...,Terminal Campina do Siqueira - Campo Comprido,-25.435724,-49.306998,NaN,NaN,0,14506.0,NaN,NaN
1,270,104905,Terminal Carmo - 030 - Interbairros III,Terminal Carmo - 030 - Interbairros III (Senti...,-25.501341,-49.237597,NaN,NaN,0,14485.0,NaN,NaN
2,276,105606,Terminal Oficinas - 030 - Interbairros III,Terminal Oficinas - 030 - Interbairros III (Se...,-25.451550,-49.214917,NaN,NaN,0,14481.0,NaN,NaN
3,299,105603,Terminal Oficinas - 030 - Interbairros III,Terminal Oficinas - 030 - Interbairros III (Se...,-25.451665,-49.215086,NaN,NaN,0,14481.0,NaN,NaN
4,308,104907,Terminal Carmo - 030 - Interbairros III,Terminal Carmo - 030 - Interbairros III (Senti...,-25.501311,-49.237825,NaN,NaN,0,14485.0,NaN,NaN


In [21]:
stops_parent_stations = stops_df[['stop_id','parent_station']]

In [22]:
otp_suggestions = otp_suggestions.merge(stops_parent_stations, left_on='from_stop_id', right_on='stop_id', how='left') \
                                    .drop(['stop_id'], axis=1)

In [23]:
otp_suggestions.head()

,date,user_trip_id,itinerary_id,leg_id,otp_start_time,otp_end_time,mode,route,from_stop_id,to_stop_id,otp_duration_mins,parent_station
0,2017-05-09,1496,1,1,2017-05-09 05:14:11,2017-05-09 05:17:49,WALK,nan,NaN,NaN,3.633333,NaN
1,2017-05-09,1496,1,2,2017-05-09 05:17:50,2017-05-09 05:45:00,BUS,272,27222,26365.0,27.166667,NaN
2,2017-05-09,1496,1,3,2017-05-09 05:45:00,2017-05-09 05:47:44,WALK,nan,NaN,NaN,2.733333,NaN
3,2017-05-09,1496,1,4,2017-05-09 05:55:00,2017-05-09 06:15:56,BUS,304,25891,26286.0,20.933333,41860.0
4,2017-05-09,1496,1,5,2017-05-09 06:15:56,2017-05-09 06:16:35,WALK,nan,NaN,NaN,0.650000,NaN


#### Selecting trips for whom OTP suggestions were found

In [24]:
selected_trips = trips_origins_clean[trips_origins_clean['o_boarding_id'].isin(otp_suggestions['user_trip_id'])]

In [25]:
len(selected_trips)

920

In [26]:
selected_trips.head()

,cardNum,o_boarding_id,o_boarding_datetime,o_route,o_busCode,o_tripNum,o_stopPointId,o_gps_datetime,o_stop_lat,o_stop_lon,...,next_o_boarding_datetime,next_o_route,next_o_busCode,next_o_tripNum,next_o_stopPointId,next_o_gps_datetime,next_o_stop_lat,next_o_stop_lon,boardings_timediff,dist_between_origins
0,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,...,2017-05-09 14:46:17,335,EA302,12.0,33577,2017-05-09 14:45:27,-25.511040,-49.224010,10:04:22,6.653455
1,3766348.0,247072,2017-05-09 14:46:17,335,EA302,12.0,33577,2017-05-09 14:45:27,-25.511040,-49.224010,...,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,10:04:22,6.653455
2,3108318.0,131140,2017-05-09 04:43:59,548,GA205,1.0,44395,2017-05-09 04:43:29,-25.558834,-49.258696,...,2017-05-09 15:36:09,000,09029,NaN,26121,NaT,-25.464630,-49.291315,10:52:10,10.974491
3,3108318.0,131141,2017-05-09 15:36:09,000,09029,NaN,26121,NaT,-25.464630,-49.291315,...,2017-05-09 04:43:59,548,GA205,1.0,44395,2017-05-09 04:43:29,-25.558834,-49.258696,10:52:10,10.974491
4,3609013.0,201240,2017-05-09 04:44:28,548,GA205,1.0,31597,2017-05-09 04:44:09,-25.560652,-49.256788,...,2017-05-09 15:07:11,021,08046,NaN,41796,NaT,-25.430289,-49.292615,10:22:43,14.935055


In [27]:
itineraries_start = otp_suggestions.query('mode == \'BUS\'') \
                    .groupby(['user_trip_id','itinerary_id']) \
                    .first() \
                    .reset_index()

In [28]:
itineraries_start.head()

,user_trip_id,itinerary_id,date,leg_id,otp_start_time,otp_end_time,mode,route,from_stop_id,to_stop_id,otp_duration_mins,parent_station
0,1496,1,2017-05-09,2,2017-05-09 05:17:50,2017-05-09 05:45:00,BUS,272,27222.0,26365.0,27.166667,41860.0
1,1496,2,2017-05-09,1,2017-05-09 05:03:32,2017-05-09 05:20:00,BUS,226,34486.0,30452.0,16.466667,14471.0
2,1496,3,2017-05-09,2,2017-05-09 05:23:03,2017-05-09 05:55:00,BUS,270,31703.0,26358.0,31.950000,41860.0
3,1496,4,2017-05-09,1,2017-05-09 05:28:22,2017-05-09 05:44:00,BUS,226,34486.0,30452.0,15.633333,14471.0
4,1496,5,2017-05-09,2,2017-05-09 05:40:25,2017-05-09 06:11:00,BUS,271,31703.0,26359.0,30.583333,41860.0


In [29]:
vehicle_boarding_origins = selected_trips[np.logical_not(selected_trips['o_busCode'].str.isdigit())]

In [30]:
len(vehicle_boarding_origins)

555

In [31]:
vehicle_boarding_origins.head()

,cardNum,o_boarding_id,o_boarding_datetime,o_route,o_busCode,o_tripNum,o_stopPointId,o_gps_datetime,o_stop_lat,o_stop_lon,...,next_o_boarding_datetime,next_o_route,next_o_busCode,next_o_tripNum,next_o_stopPointId,next_o_gps_datetime,next_o_stop_lat,next_o_stop_lon,boardings_timediff,dist_between_origins
0,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,...,2017-05-09 14:46:17,335,EA302,12.0,33577,2017-05-09 14:45:27,-25.511040,-49.224010,10:04:22,6.653455
1,3766348.0,247072,2017-05-09 14:46:17,335,EA302,12.0,33577,2017-05-09 14:45:27,-25.511040,-49.224010,...,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,10:04:22,6.653455
2,3108318.0,131140,2017-05-09 04:43:59,548,GA205,1.0,44395,2017-05-09 04:43:29,-25.558834,-49.258696,...,2017-05-09 15:36:09,000,09029,NaN,26121,NaT,-25.464630,-49.291315,10:52:10,10.974491
4,3609013.0,201240,2017-05-09 04:44:28,548,GA205,1.0,31597,2017-05-09 04:44:09,-25.560652,-49.256788,...,2017-05-09 15:07:11,021,08046,NaN,41796,NaT,-25.430289,-49.292615,10:22:43,14.935055
6,1940885.0,42210,2017-05-09 04:44:32,646,HA023,7.0,38521,2017-05-09 12:59:44,-25.565768,-49.314859,...,2017-05-09 16:11:39,TPH,03019,NaN,14494,NaT,-25.512487,-49.295048,11:27:07,6.249052


In [32]:
vehicle_boarding_origins.dtypes

cardNum                             float64
o_boarding_id                         int64
o_boarding_datetime          datetime64[ns]
o_route                              object
o_busCode                            object
o_tripNum                           float64
o_stopPointId                         int64
o_gps_datetime               datetime64[ns]
o_stop_lat                          float64
o_stop_lon                          float64
next_o_boarding_id                    int64
next_o_boarding_datetime     datetime64[ns]
next_o_route                         object
next_o_busCode                       object
next_o_tripNum                      float64
next_o_stopPointId                    int64
next_o_gps_datetime          datetime64[ns]
next_o_stop_lat                     float64
next_o_stop_lon                     float64
boardings_timediff          timedelta64[ns]
dist_between_origins                float64
dtype: object

In [33]:
itineraries_start.dtypes

user_trip_id                  int64
itinerary_id                  int64
date                 datetime64[ns]
leg_id                        int64
otp_start_time       datetime64[ns]
otp_end_time         datetime64[ns]
mode                         object
route                        object
from_stop_id                float64
to_stop_id                  float64
otp_duration_mins           float64
parent_station              float64
dtype: object

In [34]:
matched_vehicle_boardings = vehicle_boarding_origins.merge(itineraries_start, left_on=['o_boarding_id','o_route','o_stopPointId'], right_on=['user_trip_id','route','from_stop_id'], how='inner')

In [35]:
num_matched_vehicle_boardings = len(matched_vehicle_boardings.drop_duplicates(subset=['cardNum','o_boarding_id']))

print "Vehicle boardings with matching OTP suggestions: ", num_matched_vehicle_boardings, "(", \
                                                            100*(num_matched_vehicle_boardings/float(len(vehicle_boarding_origins))), "%)"

Vehicle boardings with matching OTP suggestions:  498 ( 89.7297297297 %)


In [36]:
matched_vehicle_boardings.head()

,cardNum,o_boarding_id,o_boarding_datetime,o_route,o_busCode,o_tripNum,o_stopPointId,o_gps_datetime,o_stop_lat,o_stop_lon,...,date,leg_id,otp_start_time,otp_end_time,mode,route,from_stop_id,to_stop_id,otp_duration_mins,parent_station
0,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.55871,-49.26409,...,2017-05-09,1,2017-05-09 04:41:40,2017-05-09 04:54:01,BUS,548,31593.0,31572.0,12.350000,14489.0
1,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.55871,-49.26409,...,2017-05-09,1,2017-05-09 05:07:32,2017-05-09 05:18:57,BUS,548,31593.0,31572.0,11.416667,14489.0
2,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.55871,-49.26409,...,2017-05-09,1,2017-05-09 04:55:32,2017-05-09 05:18:00,BUS,548,31593.0,4661.0,22.466667,14492.0
3,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.55871,-49.26409,...,2017-05-09,1,2017-05-09 05:19:32,2017-05-09 05:42:00,BUS,548,31593.0,4661.0,22.466667,14492.0
4,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.55871,-49.26409,...,2017-05-09,1,2017-05-09 05:31:32,2017-05-09 05:42:57,BUS,548,31593.0,31572.0,11.416667,NaN


In [37]:
pd.concat([matched_vehicle_boardings[vehicle_boarding_origins.columns],vehicle_boarding_origins]).drop_duplicates(keep=False)

,cardNum,o_boarding_id,o_boarding_datetime,o_route,o_busCode,o_tripNum,o_stopPointId,o_gps_datetime,o_stop_lat,o_stop_lon,...,next_o_boarding_datetime,next_o_route,next_o_busCode,next_o_tripNum,next_o_stopPointId,next_o_gps_datetime,next_o_stop_lat,next_o_stop_lon,boardings_timediff,dist_between_origins
6,1940885.0,42210,2017-05-09 04:44:32,646,HA023,7.0,38521,2017-05-09 12:59:44,-25.565768,-49.314859,...,2017-05-09 16:11:39,TPH,03019,NaN,14494,NaT,-25.512487,-49.295048,11:27:07,6.249052
21,2178773.0,54396,2017-05-09 04:46:13,619,HA600,5.0,30434,2017-05-09 12:57:05,-25.511661,-49.324223,...,2017-05-09 05:57:54,000,03044,NaN,41826,NaT,-25.439459,-49.271565,01:11:41,9.612407
43,3491154.0,178753,2017-05-09 05:54:25,967,MN600,1.0,33734,2017-05-09 05:49:32,-25.426369,-49.277005,...,2017-05-09 15:10:03,967,MN604,10.0,33397,2017-05-09 15:08:47,-25.408069,-49.304026,09:15:38,3.391939
96,3510903.0,182201,2017-05-09 05:17:29,040,HB606,1.0,35108,2017-05-09 05:16:53,-25.512566,-49.330333,...,2017-05-09 04:48:58,619,HA600,1.0,36284,2017-05-09 04:48:47,-25.573642,-49.332666,00:28:31,6.795384
148,2258052.0,60146,2017-05-09 18:44:40,870,BC189,10.0,29420,2017-05-09 18:39:00,-25.430894,-49.276454,...,2017-05-09 04:51:41,040,MB604,1.0,34260,2017-05-09 04:50:48,-25.423548,-49.351123,13:52:59,7.542864
150,3620341.0,204008,2017-05-09 05:54:30,967,MN600,1.0,33734,2017-05-09 05:49:32,-25.426369,-49.277005,...,2017-05-09 14:20:26,901,MC076,9.0,33676,2017-05-09 14:20:13,-25.415973,-49.296807,08:25:56,2.300296
219,2113177.0,50896,2017-05-09 04:53:48,643,GA003,7.0,38025,2017-05-09 12:59:43,-25.585504,-49.274348,...,2017-05-09 05:16:13,000,09048,NaN,41829,NaT,-25.527912,-49.291347,00:22:25,6.627091
257,3207482.0,140017,2017-05-09 04:55:08,542,GA124,1.0,31262,2017-05-09 04:54:19,-25.544890,-49.265160,...,2017-05-09 16:08:12,000,03041,NaN,26095,NaT,-25.443355,-49.281734,11:13:04,11.412121
264,3742409.0,238100,2017-05-09 14:50:44,650,HA030,9.0,39826,2017-05-09 14:49:47,-25.549416,-49.324263,...,2017-05-09 15:42:33,TPH,03019,NaN,14494,NaT,-25.512487,-49.295048,00:51:49,5.045196
304,1342841.0,18334,2017-05-09 04:56:09,542,GA124,1.0,31263,2017-05-09 04:56:00,-25.541510,-49.270560,...,2017-05-09 16:04:44,021,08044,NaN,41791,NaT,-25.431951,-49.296491,11:08:35,12.457360


In [38]:
itineraries_start[itineraries_start['user_trip_id'] == 60146]

,user_trip_id,itinerary_id,date,leg_id,otp_start_time,otp_end_time,mode,route,from_stop_id,to_stop_id,otp_duration_mins,parent_station
2916,60146,1,2017-05-09,2,2017-05-09 19:04:27,2017-05-09 19:33:29,BUS,870,28604.0,34258.0,29.033333,NaN
2917,60146,2,2017-05-09,2,2017-05-09 18:55:49,2017-05-09 19:27:52,BUS,876,28604.0,30333.0,32.050000,NaN
2918,60146,3,2017-05-09,2,2017-05-09 18:56:09,2017-05-09 19:10:16,BUS,303,25557.0,70.0,14.116667,41768.0
2919,60146,4,2017-05-09,2,2017-05-09 19:21:27,2017-05-09 19:50:29,BUS,870,28604.0,34258.0,29.033333,NaN
2920,60146,5,2017-05-09,2,2017-05-09 18:53:41,2017-05-09 19:35:29,BUS,972,26720.0,33342.0,41.800000,NaN
2921,60146,6,2017-05-09,2,2017-05-09 19:38:27,2017-05-09 20:07:29,BUS,870,28604.0,34258.0,29.033333,NaN
2922,60146,7,2017-05-09,2,2017-05-09 19:55:34,2017-05-09 20:27:01,BUS,870,28604.0,34258.0,31.450000,NaN
2923,60146,8,2017-05-09,2,2017-05-09 19:56:31,2017-05-09 20:23:13,BUS,876,28604.0,30333.0,26.700000,NaN
2924,60146,9,2017-05-09,2,2017-05-09 20:31:12,2017-05-09 20:55:24,BUS,870,28604.0,34258.0,24.200000,NaN
2925,60146,10,2017-05-09,2,2017-05-09 20:51:12,2017-05-09 21:15:24,BUS,870,28604.0,34258.0,24.200000,NaN


In [39]:
terminal_boarding_origins = selected_trips[selected_trips['o_busCode'].str.isdigit()]

In [40]:
terminal_boarding_origins['o_route'].value_counts()

000    303
021     37
TPH      9
TMA      4
TCJ      2
TFA      2
TCR      2
TSP      2
TCA      1
TSQ      1
TSB      1
TSF      1
Name: o_route, dtype: int64

In [41]:
terminal_021_origins = selected_trips[(selected_trips['o_busCode'].str.isdigit()) & (selected_trips['o_route'] == '021')]

In [42]:
terminal_021_origins.head()

,cardNum,o_boarding_id,o_boarding_datetime,o_route,o_busCode,o_tripNum,o_stopPointId,o_gps_datetime,o_stop_lat,o_stop_lon,...,next_o_boarding_datetime,next_o_route,next_o_busCode,next_o_tripNum,next_o_stopPointId,next_o_gps_datetime,next_o_stop_lat,next_o_stop_lon,boardings_timediff,dist_between_origins
5,3609013.0,201241,2017-05-09 15:07:11,021,08046,NaN,41796,NaT,-25.430289,-49.292615,...,2017-05-09 04:44:28,548,GA205,1.0,31597,2017-05-09 04:44:09,-25.560652,-49.256788,10:22:43,14.935055
68,1598789.0,27398,2017-05-09 16:39:51,021,08042,NaN,41765,NaT,-25.433218,-49.299969,...,2017-05-09 04:47:57,655,HA257,9.0,39262,2017-05-09 12:59:51,-25.568646,-49.319413,11:51:54,15.184799
74,1886552.0,39744,2017-05-09 16:00:02,021,04000,NaN,14506,NaT,-25.436086,-49.307596,...,2017-05-09 04:48:22,650,HA243,1.0,36293,2017-05-09 04:47:57,-25.568989,-49.332253,11:11:40,14.983937
92,2385286.0,76681,2017-05-09 15:07:35,021,08046,NaN,41796,NaT,-25.430289,-49.292615,...,2017-05-09 04:48:56,652,LA007,1.0,36697,2017-05-09 04:48:53,-25.532192,-49.337116,10:18:39,12.179731
156,1163918.0,12638,2017-05-09 14:32:14,021,08033,NaN,41760,NaT,-25.437598,-49.324377,...,2017-05-09 04:51:59,684,HA602,1.0,39377,2017-05-09 04:51:51,-25.586030,-49.334810,09:40:15,16.538020


In [43]:
matched_021_terminal_boardings = terminal_021_origins.merge(itineraries_start, left_on=['o_boarding_id','o_route','o_stopPointId'], right_on=['user_trip_id','route','parent_station'], how='inner')

In [44]:
num_matched_021_terminal_boardings = len(matched_021_terminal_boardings.drop_duplicates(subset=['cardNum','o_boarding_id']))

print "Line 021 Terminal boardings with matching OTP suggestions: ", num_matched_021_terminal_boardings, "(", \
                                                            100*(num_matched_021_terminal_boardings/float(len(terminal_021_origins))), "%)"

Line 021 Terminal boardings with matching OTP suggestions:  1 ( 2.7027027027 %)


In [45]:
matched_021_terminal_boardings.head()

,cardNum,o_boarding_id,o_boarding_datetime,o_route,o_busCode,o_tripNum,o_stopPointId,o_gps_datetime,o_stop_lat,o_stop_lon,...,date,leg_id,otp_start_time,otp_end_time,mode,route,from_stop_id,to_stop_id,otp_duration_mins,parent_station
0,1886552.0,39744,2017-05-09 16:00:02,021,04000,NaN,14506,NaT,-25.436086,-49.307596,...,2017-05-09,2,2017-05-09 16:04:00,2017-05-09 16:36:00,BUS,021,28136.0,27475.0,32.00,14506.0
1,1886552.0,39744,2017-05-09 16:00:02,021,04000,NaN,14506,NaT,-25.436086,-49.307596,...,2017-05-09,2,2017-05-09 16:18:00,2017-05-09 16:45:21,BUS,021,28136.0,33044.0,27.35,14506.0
2,1886552.0,39744,2017-05-09 16:00:02,021,04000,NaN,14506,NaT,-25.436086,-49.307596,...,2017-05-09,2,2017-05-09 16:27:00,2017-05-09 16:57:00,BUS,021,28136.0,27475.0,30.00,14506.0


In [46]:
terminal_boarding_origins = selected_trips[(selected_trips['o_busCode'].str.isdigit()) & (selected_trips['o_route'] != '021')]

In [47]:
terminal_boarding_origins.head()

,cardNum,o_boarding_id,o_boarding_datetime,o_route,o_busCode,o_tripNum,o_stopPointId,o_gps_datetime,o_stop_lat,o_stop_lon,...,next_o_boarding_datetime,next_o_route,next_o_busCode,next_o_tripNum,next_o_stopPointId,next_o_gps_datetime,next_o_stop_lat,next_o_stop_lon,boardings_timediff,dist_between_origins
3,3108318.0,131141,2017-05-09 15:36:09,000,09029,NaN,26121,NaT,-25.464630,-49.291315,...,2017-05-09 04:43:59,548,GA205,1.0,44395,2017-05-09 04:43:29,-25.558834,-49.258696,10:52:10,10.974491
7,1940885.0,42211,2017-05-09 16:11:39,TPH,03019,NaN,14494,NaT,-25.512487,-49.295048,...,2017-05-09 04:44:32,646,HA023,7.0,38521,2017-05-09 12:59:44,-25.565768,-49.314859,11:27:07,6.249052
9,1232958.0,14668,2017-05-09 15:04:54,000,00024,NaN,43347,NaT,-25.475898,-49.249983,...,2017-05-09 04:45:04,548,GA205,1.0,48762,2017-05-09 04:44:52,-25.563193,-49.253670,10:19:50,9.713823
11,715315.0,2902,2017-05-09 13:33:44,000,09070,NaN,43723,NaT,-25.433715,-49.270258,...,2017-05-09 04:45:14,684,HA602,1.0,39368,2017-05-09 04:43:41,-25.588450,-49.341190,08:48:30,18.620097
13,1982410.0,44255,2017-05-09 18:38:20,000,00070,NaN,14487,NaT,-25.481927,-49.246999,...,2017-05-09 04:45:28,684,HA602,1.0,39369,2017-05-09 04:45:27,-25.592110,-49.342120,13:52:52,15.530264


In [48]:
matched_terminal_boardings = terminal_boarding_origins.merge(itineraries_start, left_on=['o_boarding_id','o_stopPointId'], right_on=['user_trip_id','parent_station'], how='inner') \
                .drop_duplicates(subset=['cardNum','o_boarding_id'])

In [49]:
num_matched_terminal_boardings = len(matched_terminal_boardings.drop_duplicates(subset=['cardNum','o_boarding_id']))

print "Terminal boardings with matching OTP suggestions: ", num_matched_terminal_boardings, "(", \
                                                            100*(num_matched_terminal_boardings/float(len(terminal_boarding_origins))), "%)"

Terminal boardings with matching OTP suggestions:  226 ( 68.9024390244 %)


In [50]:
matched_terminal_boardings.head()

,cardNum,o_boarding_id,o_boarding_datetime,o_route,o_busCode,o_tripNum,o_stopPointId,o_gps_datetime,o_stop_lat,o_stop_lon,...,date,leg_id,otp_start_time,otp_end_time,mode,route,from_stop_id,to_stop_id,otp_duration_mins,parent_station
0,3108318.0,131141,2017-05-09 15:36:09,000,09029,NaN,26121,NaT,-25.464630,-49.291315,...,2017-05-09,2,2017-05-09 15:41:31,2017-05-09 16:06:00,BUS,603,25398.0,26296.0,24.483333,26121.0
9,1940885.0,42211,2017-05-09 16:11:39,TPH,03019,NaN,14494,NaT,-25.512487,-49.295048,...,2017-05-09,1,2017-05-09 16:13:00,2017-05-09 16:33:37,BUS,617,31051.0,38521.0,20.616667,14494.0
19,1232958.0,14668,2017-05-09 15:04:54,000,00024,NaN,43347,NaT,-25.475898,-49.249983,...,2017-05-09,2,2017-05-09 15:05:39,2017-05-09 15:08:41,BUS,602,25468.0,25463.0,3.033333,43347.0
23,1982410.0,44255,2017-05-09 18:38:20,000,00070,NaN,14487,NaT,-25.481927,-49.246999,...,2017-05-09,2,2017-05-09 18:40:00,2017-05-09 18:45:41,BUS,602,25474.0,25463.0,5.683333,14487.0
33,2178773.0,54398,2017-05-09 15:41:51,000,06057,NaN,26011,NaT,-25.435634,-49.256611,...,2017-05-09,2,2017-05-09 15:48:26,2017-05-09 15:55:00,BUS,303,25646.0,26185.0,6.566667,26011.0


In [51]:
total_num_matches = num_matched_vehicle_boardings + num_matched_021_terminal_boardings + num_matched_terminal_boardings

print "Total number of matches: ", total_num_matches, "(", \
                                    100*(total_num_matches/float(len(selected_trips))), "%)"

Total number of matches:  725 ( 78.8043478261 %)


In [52]:
boarding_suggestions_matches = pd.concat([matched_vehicle_boardings,matched_021_terminal_boardings,matched_terminal_boardings])

In [53]:
len(boarding_suggestions_matches)

2298

In [54]:
boarding_suggestions_matches.head()

,cardNum,o_boarding_id,o_boarding_datetime,o_route,o_busCode,o_tripNum,o_stopPointId,o_gps_datetime,o_stop_lat,o_stop_lon,...,date,leg_id,otp_start_time,otp_end_time,mode,route,from_stop_id,to_stop_id,otp_duration_mins,parent_station
0,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.55871,-49.26409,...,2017-05-09,1,2017-05-09 04:41:40,2017-05-09 04:54:01,BUS,548,31593.0,31572.0,12.350000,14489.0
1,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.55871,-49.26409,...,2017-05-09,1,2017-05-09 05:07:32,2017-05-09 05:18:57,BUS,548,31593.0,31572.0,11.416667,14489.0
2,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.55871,-49.26409,...,2017-05-09,1,2017-05-09 04:55:32,2017-05-09 05:18:00,BUS,548,31593.0,4661.0,22.466667,14492.0
3,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.55871,-49.26409,...,2017-05-09,1,2017-05-09 05:19:32,2017-05-09 05:42:00,BUS,548,31593.0,4661.0,22.466667,14492.0
4,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.55871,-49.26409,...,2017-05-09,1,2017-05-09 05:31:32,2017-05-09 05:42:57,BUS,548,31593.0,31572.0,11.416667,NaN


#### Add OTP extra origin/next-origin pairs to final dataset

In [55]:
boarding_suggestions_matches.dtypes

cardNum                             float64
o_boarding_id                         int64
o_boarding_datetime          datetime64[ns]
o_route                              object
o_busCode                            object
o_tripNum                           float64
o_stopPointId                        object
o_gps_datetime               datetime64[ns]
o_stop_lat                          float64
o_stop_lon                          float64
next_o_boarding_id                    int64
next_o_boarding_datetime     datetime64[ns]
next_o_route                         object
next_o_busCode                       object
next_o_tripNum                      float64
next_o_stopPointId                    int64
next_o_gps_datetime          datetime64[ns]
next_o_stop_lat                     float64
next_o_stop_lon                     float64
boardings_timediff          timedelta64[ns]
dist_between_origins                float64
user_trip_id                          int64
itinerary_id                    

In [56]:
all_boarding_suggestions_matches = boarding_suggestions_matches[np.append(trips_origins_clean.columns.values,['itinerary_id'])] \
                    .merge(otp_suggestions, left_on=['o_boarding_id','itinerary_id'], right_on=['user_trip_id','itinerary_id'], how='inner') \
                    .query('mode == \'BUS\'')

In [57]:
len(all_boarding_suggestions_matches)

5231

In [58]:
all_boarding_suggestions_matches

,cardNum,o_boarding_id,o_boarding_datetime,o_route,o_busCode,o_tripNum,o_stopPointId,o_gps_datetime,o_stop_lat,o_stop_lon,...,user_trip_id,leg_id,otp_start_time,otp_end_time,mode,route,from_stop_id,to_stop_id,otp_duration_mins,parent_station
0,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,...,247071,1,2017-05-09 04:41:40,2017-05-09 04:54:01,BUS,548,31593,31572.0,12.350000,NaN
2,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,...,247071,3,2017-05-09 05:04:44,2017-05-09 05:22:00,BUS,535,31571,30030.0,17.266667,NaN
4,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,...,247071,5,2017-05-09 05:30:00,2017-05-09 05:33:55,BUS,335,30027,33576.0,3.916667,14489.0
6,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,...,247071,1,2017-05-09 05:07:32,2017-05-09 05:18:57,BUS,548,31593,31572.0,11.416667,NaN
8,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,...,247071,3,2017-05-09 05:19:19,2017-05-09 05:35:00,BUS,535,31571,30030.0,15.683333,NaN
10,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,...,247071,5,2017-05-09 05:45:00,2017-05-09 05:48:08,BUS,335,30027,33576.0,3.133333,14489.0
12,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,...,247071,1,2017-05-09 04:55:32,2017-05-09 05:18:00,BUS,548,31593,4661.0,22.466667,NaN
14,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,...,247071,3,2017-05-09 05:30:00,2017-05-09 05:45:00,BUS,602,27121,27352.0,15.000000,14492.0
16,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,...,247071,1,2017-05-09 05:19:32,2017-05-09 05:42:00,BUS,548,31593,4661.0,22.466667,NaN
18,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,...,247071,3,2017-05-09 05:48:00,2017-05-09 06:01:00,BUS,508,26257,30025.0,13.000000,14492.0


In [59]:
all_boarding_suggestions_matches['first_vehicle_boarding'] = np.where(np.logical_not(all_boarding_suggestions_matches['o_busCode'].str.isdigit()) & 
                                                                      (all_boarding_suggestions_matches['o_route'] == all_boarding_suggestions_matches['route']),
                                                                      True,
                                                                      False)

In [60]:
all_boarding_suggestions_matches

,cardNum,o_boarding_id,o_boarding_datetime,o_route,o_busCode,o_tripNum,o_stopPointId,o_gps_datetime,o_stop_lat,o_stop_lon,...,leg_id,otp_start_time,otp_end_time,mode,route,from_stop_id,to_stop_id,otp_duration_mins,parent_station,first_vehicle_boarding
0,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,...,1,2017-05-09 04:41:40,2017-05-09 04:54:01,BUS,548,31593,31572.0,12.350000,NaN,True
2,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,...,3,2017-05-09 05:04:44,2017-05-09 05:22:00,BUS,535,31571,30030.0,17.266667,NaN,False
4,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,...,5,2017-05-09 05:30:00,2017-05-09 05:33:55,BUS,335,30027,33576.0,3.916667,14489.0,False
6,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,...,1,2017-05-09 05:07:32,2017-05-09 05:18:57,BUS,548,31593,31572.0,11.416667,NaN,True
8,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,...,3,2017-05-09 05:19:19,2017-05-09 05:35:00,BUS,535,31571,30030.0,15.683333,NaN,False
10,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,...,5,2017-05-09 05:45:00,2017-05-09 05:48:08,BUS,335,30027,33576.0,3.133333,14489.0,False
12,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,...,1,2017-05-09 04:55:32,2017-05-09 05:18:00,BUS,548,31593,4661.0,22.466667,NaN,True
14,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,...,3,2017-05-09 05:30:00,2017-05-09 05:45:00,BUS,602,27121,27352.0,15.000000,14492.0,False
16,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,...,1,2017-05-09 05:19:32,2017-05-09 05:42:00,BUS,548,31593,4661.0,22.466667,NaN,True
18,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,...,3,2017-05-09 05:48:00,2017-05-09 06:01:00,BUS,508,26257,30025.0,13.000000,14492.0,False


In [61]:
otp_filtered_itineraries = all_boarding_suggestions_matches[np.append(otp_suggestions.columns.values,['first_vehicle_boarding','o_busCode','o_tripNum','o_boarding_datetime'])]

In [62]:
otp_filtered_itineraries

,date,user_trip_id,itinerary_id,leg_id,otp_start_time,otp_end_time,mode,route,from_stop_id,to_stop_id,otp_duration_mins,parent_station,first_vehicle_boarding,o_busCode,o_tripNum,o_boarding_datetime
0,2017-05-09,247071,1,1,2017-05-09 04:41:40,2017-05-09 04:54:01,BUS,548,31593,31572.0,12.350000,NaN,True,GA205,1.0,2017-05-09 04:41:55
2,2017-05-09,247071,1,3,2017-05-09 05:04:44,2017-05-09 05:22:00,BUS,535,31571,30030.0,17.266667,NaN,False,GA205,1.0,2017-05-09 04:41:55
4,2017-05-09,247071,1,5,2017-05-09 05:30:00,2017-05-09 05:33:55,BUS,335,30027,33576.0,3.916667,14489.0,False,GA205,1.0,2017-05-09 04:41:55
6,2017-05-09,247071,2,1,2017-05-09 05:07:32,2017-05-09 05:18:57,BUS,548,31593,31572.0,11.416667,NaN,True,GA205,1.0,2017-05-09 04:41:55
8,2017-05-09,247071,2,3,2017-05-09 05:19:19,2017-05-09 05:35:00,BUS,535,31571,30030.0,15.683333,NaN,False,GA205,1.0,2017-05-09 04:41:55
10,2017-05-09,247071,2,5,2017-05-09 05:45:00,2017-05-09 05:48:08,BUS,335,30027,33576.0,3.133333,14489.0,False,GA205,1.0,2017-05-09 04:41:55
12,2017-05-09,247071,3,1,2017-05-09 04:55:32,2017-05-09 05:18:00,BUS,548,31593,4661.0,22.466667,NaN,True,GA205,1.0,2017-05-09 04:41:55
14,2017-05-09,247071,3,3,2017-05-09 05:30:00,2017-05-09 05:45:00,BUS,602,27121,27352.0,15.000000,14492.0,False,GA205,1.0,2017-05-09 04:41:55
16,2017-05-09,247071,5,1,2017-05-09 05:19:32,2017-05-09 05:42:00,BUS,548,31593,4661.0,22.466667,NaN,True,GA205,1.0,2017-05-09 04:41:55
18,2017-05-09,247071,5,3,2017-05-09 05:48:00,2017-05-09 06:01:00,BUS,508,26257,30025.0,13.000000,14492.0,False,GA205,1.0,2017-05-09 04:41:55


#### Find OTP Suggested Itineraries in BUSTE Data

In [63]:
bus_trips = pd.read_csv('/local/tarciso/masters/data/bus_trips/test/enhanced-buste/2017_05_09_bus_trips.csv', parse_dates=['gps_datetime'])

/local/tarciso/programs/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [64]:
bus_trips = bus_trips.sort_values(['route','busCode','tripNum','gps_datetime'])
bus_trips['route'] = bus_trips['route'].astype(str).str.replace("\.0",'').str.zfill(3)

In [65]:
bus_trips.head()

,route,busCode,tripNum,stopPointId,gps_datetime,stop_lat,stop_lon,parent_station
139946,001,BN997,1.0,29887,2017-05-09 06:47:54,-25.428171,-49.264558,NaN
139947,001,BN997,1.0,31453,2017-05-09 06:48:54,-25.430388,-49.263602,NaN
138395,001,BN997,1.0,31454,2017-05-09 06:49:52,-25.433503,-49.262257,NaN
139948,001,BN997,1.0,30748,2017-05-09 06:52:21,-25.435187,-49.264933,NaN
139949,001,BN997,1.0,30749,2017-05-09 06:52:35,-25.435959,-49.266889,NaN


##### Vehicle Boarding (1st boarding)

In [66]:
otp_vehicle_first_boarding = otp_filtered_itineraries.query('first_vehicle_boarding == True') \
                                .sort_values(['user_trip_id','itinerary_id'])

In [67]:
len(otp_vehicle_first_boarding)

2080

In [68]:
otp_vehicle_first_boarding

,date,user_trip_id,itinerary_id,leg_id,otp_start_time,otp_end_time,mode,route,from_stop_id,to_stop_id,otp_duration_mins,parent_station,first_vehicle_boarding,o_busCode,o_tripNum,o_boarding_datetime
6634,2017-05-09,1496,2,1,2017-05-09 05:03:32,2017-05-09 05:20:00,BUS,226,34486,30452.0,16.466667,NaN,True,BA140,1.0,2017-05-09 05:04:34
6641,2017-05-09,1496,4,1,2017-05-09 05:28:22,2017-05-09 05:44:00,BUS,226,34486,30452.0,15.633333,NaN,True,BA140,1.0,2017-05-09 05:04:34
5689,2017-05-09,1869,2,1,2017-05-09 05:03:02,2017-05-09 05:20:00,BUS,653,36174,30435.0,16.966667,NaN,True,HA270,1.0,2017-05-09 05:03:17
5694,2017-05-09,1869,4,1,2017-05-09 05:23:02,2017-05-09 05:40:00,BUS,653,36174,30435.0,16.966667,NaN,True,HA270,1.0,2017-05-09 05:03:17
5699,2017-05-09,1869,5,1,2017-05-09 05:31:05,2017-05-09 05:48:25,BUS,653,36174,35104.0,17.333333,NaN,True,HA270,1.0,2017-05-09 05:03:17
5706,2017-05-09,1869,7,1,2017-05-09 05:43:02,2017-05-09 06:00:00,BUS,653,36174,30435.0,16.966667,NaN,True,HA270,1.0,2017-05-09 05:03:17
5711,2017-05-09,1869,9,1,2017-05-09 05:51:14,2017-05-09 06:00:21,BUS,653,36174,33841.0,9.116667,NaN,True,HA270,1.0,2017-05-09 05:03:17
5719,2017-05-09,1870,6,1,2017-05-09 16:07:57,2017-05-09 16:18:00,BUS,216,29126,28128.0,10.050000,NaN,True,BA600,6.0,2017-05-09 15:56:24
4884,2017-05-09,1947,1,1,2017-05-09 05:07:18,2017-05-09 05:20:00,BUS,639,32033,31026.0,12.700000,NaN,True,GA177,1.0,2017-05-09 05:01:08
4888,2017-05-09,1947,5,1,2017-05-09 05:20:00,2017-05-09 05:31:00,BUS,639,32033,31026.0,11.000000,NaN,True,GA177,1.0,2017-05-09 05:01:08


In [69]:
actual_otp_vehicle_first_boarding = otp_vehicle_first_boarding.merge(bus_trips, 
                                 left_on=['route','o_busCode','o_tripNum','to_stop_id'], 
                                 right_on=['route','busCode','tripNum','stopPointId'], 
                                 how='inner') \
                            .sort_values(['user_trip_id','itinerary_id','gps_datetime']) \
                            .groupby(['user_trip_id','itinerary_id','leg_id']) \
                            .first() \
                            .reset_index()                            

In [70]:
len(actual_otp_vehicle_first_boarding)

2009

In [71]:
actual_otp_vehicle_first_boarding.head()

,user_trip_id,itinerary_id,leg_id,date,otp_start_time,otp_end_time,mode,route,from_stop_id,to_stop_id,...,o_busCode,o_tripNum,o_boarding_datetime,busCode,tripNum,stopPointId,gps_datetime,stop_lat,stop_lon,parent_station_y
0,1496,2,1,2017-05-09,2017-05-09 05:03:32,2017-05-09 05:20:00,BUS,226,34486.0,30452.0,...,BA140,1.0,2017-05-09 05:04:34,BA140,1.0,30452,2017-05-09 05:14:02,-25.393259,-49.240989,14471.0
1,1496,4,1,2017-05-09,2017-05-09 05:28:22,2017-05-09 05:44:00,BUS,226,34486.0,30452.0,...,BA140,1.0,2017-05-09 05:04:34,BA140,1.0,30452,2017-05-09 05:14:02,-25.393259,-49.240989,14471.0
2,1869,2,1,2017-05-09,2017-05-09 05:03:02,2017-05-09 05:20:00,BUS,653,36174.0,30435.0,...,HA270,1.0,2017-05-09 05:03:17,HA270,1.0,30435,2017-05-09 05:17:17,-25.511621,-49.324356,14503.0
3,1869,4,1,2017-05-09,2017-05-09 05:23:02,2017-05-09 05:40:00,BUS,653,36174.0,30435.0,...,HA270,1.0,2017-05-09 05:03:17,HA270,1.0,30435,2017-05-09 05:17:17,-25.511621,-49.324356,14503.0
4,1869,5,1,2017-05-09,2017-05-09 05:31:05,2017-05-09 05:48:25,BUS,653,36174.0,35104.0,...,HA270,1.0,2017-05-09 05:03:17,HA270,1.0,35104,2017-05-09 05:16:11,-25.511630,-49.328224,NaN


##### Terminal Boarding and Vehicle Boarding (2nd+ boarding)

In [72]:
otp_other_boardings = otp_filtered_itineraries.query('first_vehicle_boarding == False')

In [73]:
otp_filtered_itineraries.head()

,date,user_trip_id,itinerary_id,leg_id,otp_start_time,otp_end_time,mode,route,from_stop_id,to_stop_id,otp_duration_mins,parent_station,first_vehicle_boarding,o_busCode,o_tripNum,o_boarding_datetime
0,2017-05-09,247071,1,1,2017-05-09 04:41:40,2017-05-09 04:54:01,BUS,548,31593,31572.0,12.350000,NaN,True,GA205,1.0,2017-05-09 04:41:55
2,2017-05-09,247071,1,3,2017-05-09 05:04:44,2017-05-09 05:22:00,BUS,535,31571,30030.0,17.266667,NaN,False,GA205,1.0,2017-05-09 04:41:55
4,2017-05-09,247071,1,5,2017-05-09 05:30:00,2017-05-09 05:33:55,BUS,335,30027,33576.0,3.916667,14489.0,False,GA205,1.0,2017-05-09 04:41:55
6,2017-05-09,247071,2,1,2017-05-09 05:07:32,2017-05-09 05:18:57,BUS,548,31593,31572.0,11.416667,NaN,True,GA205,1.0,2017-05-09 04:41:55
8,2017-05-09,247071,2,3,2017-05-09 05:19:19,2017-05-09 05:35:00,BUS,535,31571,30030.0,15.683333,NaN,False,GA205,1.0,2017-05-09 04:41:55


In [97]:
actual_otp_filtered_itineraries_start = otp_filtered_itineraries.merge(bus_trips, 
                                 left_on=['route','from_stop_id'], 
                                 right_on=['route','stopPointId'], 
                                 how='inner') \
                        .assign(otp_buste_start_timediff = lambda x: np.absolute(x['gps_datetime'] - x['otp_start_time'])) \
                        .sort_values(['user_trip_id','itinerary_id','leg_id','otp_buste_start_timediff']) \
                        .groupby(['user_trip_id','itinerary_id','leg_id']) \
                        .first() \
                        .reset_index() \
                        .loc[:, ['user_trip_id','first_vehicle_boarding','itinerary_id','leg_id','route','busCode','o_busCode','tripNum','o_tripNum','from_stop_id','otp_start_time','gps_datetime','o_boarding_datetime','otp_buste_start_timediff','to_stop_id','otp_end_time']]

In [98]:
len(actual_otp_filtered_itineraries_start)

5195

In [99]:
actual_otp_filtered_itineraries_start.head()

,user_trip_id,first_vehicle_boarding,itinerary_id,leg_id,route,busCode,o_busCode,tripNum,o_tripNum,from_stop_id,otp_start_time,gps_datetime,o_boarding_datetime,otp_buste_start_timediff,to_stop_id,otp_end_time
0,1496,True,2,1,226,BA140,BA140,1.0,1.0,34486.0,2017-05-09 05:03:32,2017-05-09 05:03:32,2017-05-09 05:04:34,00:00:00,30452.0,2017-05-09 05:20:00
1,1496,False,2,3,203,BD131,BA140,1.0,1.0,26190.0,2017-05-09 05:25:46,2017-05-09 05:26:08,2017-05-09 05:04:34,00:00:22,25389.0,2017-05-09 05:45:54
2,1496,False,2,4,303,DE701,BA140,1.0,1.0,25389.0,2017-05-09 05:49:21,2017-05-09 05:44:38,2017-05-09 05:04:34,00:04:43,26195.0,2017-05-09 06:20:00
3,1496,False,2,6,732,JA024,BA140,1.0,1.0,31002.0,2017-05-09 06:23:00,2017-05-09 06:18:42,2017-05-09 05:04:34,00:04:18,33185.0,2017-05-09 06:30:19
4,1496,True,4,1,226,BA140,BA140,2.0,1.0,34486.0,2017-05-09 05:28:22,2017-05-09 05:45:57,2017-05-09 05:04:34,00:17:35,30452.0,2017-05-09 05:44:00


In [82]:
actual_otp_filtered_itineraries_start.otp_buste_start_timediff.describe()

count                      5195
mean     0 days 00:08:49.971126
std      0 days 00:31:08.455207
min             0 days 00:00:00
25%             0 days 00:00:46
50%             0 days 00:02:04
75%      0 days 00:05:38.500000
max             0 days 16:33:48
Name: otp_buste_start_timediff, dtype: object

In [226]:
actual_otp_filtered_itineraries = actual_otp_filtered_itineraries_start \
                        .rename(index=str, columns={'to_stop_id':'stopPointId', 'gps_datetime':'matched_start_time'}) \
                        .merge(bus_trips, 
                                 on=['route','busCode','tripNum','stopPointId'], 
                                 how='inner') \
                        .assign(otp_buste_end_timediff = lambda x: np.absolute(x['gps_datetime'] - x['otp_end_time'])) \
                        .sort_values(['user_trip_id','itinerary_id','leg_id','otp_buste_end_timediff']) \
                        .groupby(['user_trip_id','itinerary_id','leg_id']) \
                        .first() \
                        .reset_index() \
                        .rename(index=str, columns={'stopPointId':'to_stop_id', 'gps_datetime':'matched_end_time'}) \
                        .assign(leg_duration = lambda x: x['matched_end_time'] - x['matched_start_time'],
                                boarding_otp_match_start_timediff = lambda x: np.absolute(x['o_boarding_datetime'] - x['matched_start_time'])) \
                        .loc[:, ['user_trip_id','first_vehicle_boarding','itinerary_id','leg_id','route','busCode','o_busCode','tripNum','o_tripNum','from_stop_id','otp_start_time','matched_start_time','o_boarding_datetime','otp_buste_start_timediff','to_stop_id','otp_end_time','matched_end_time','otp_buste_end_timediff', 'boarding_otp_match_start_timediff', 'leg_duration']]
                        

In [227]:
len(actual_otp_filtered_itineraries)

4985

In [228]:
actual_otp_filtered_itineraries.head(20)

,user_trip_id,first_vehicle_boarding,itinerary_id,leg_id,route,busCode,o_busCode,tripNum,o_tripNum,from_stop_id,otp_start_time,matched_start_time,o_boarding_datetime,otp_buste_start_timediff,to_stop_id,otp_end_time,matched_end_time,otp_buste_end_timediff,boarding_otp_match_start_timediff,leg_duration
0,1496,True,2,1,226,BA140,BA140,1.0,1.0,34486.0,2017-05-09 05:03:32,2017-05-09 05:03:32,2017-05-09 05:04:34,00:00:00,30452.0,2017-05-09 05:20:00,2017-05-09 05:14:02,00:05:58,00:01:02,00:10:30
1,1496,False,2,3,203,BD131,BA140,1.0,1.0,26190.0,2017-05-09 05:25:46,2017-05-09 05:26:08,2017-05-09 05:04:34,00:00:22,25389.0,2017-05-09 05:45:54,2017-05-09 05:47:45,00:01:51,00:21:34,00:21:37
2,1496,False,2,4,303,DE701,BA140,1.0,1.0,25389.0,2017-05-09 05:49:21,2017-05-09 05:44:38,2017-05-09 05:04:34,00:04:43,26195.0,2017-05-09 06:20:00,2017-05-09 06:14:56,00:05:04,00:40:04,00:30:18
3,1496,False,2,6,732,JA024,BA140,1.0,1.0,31002.0,2017-05-09 06:23:00,2017-05-09 06:18:42,2017-05-09 05:04:34,00:04:18,33185.0,2017-05-09 06:30:19,2017-05-09 06:29:58,00:00:21,01:14:08,00:11:16
4,1496,True,4,1,226,BA140,BA140,2.0,1.0,34486.0,2017-05-09 05:28:22,2017-05-09 05:45:57,2017-05-09 05:04:34,00:17:35,30452.0,2017-05-09 05:44:00,2017-05-09 06:03:33,00:19:33,00:41:23,00:17:36
5,1496,False,4,3,203,BD133,BA140,1.0,1.0,26190.0,2017-05-09 05:48:46,2017-05-09 05:48:38,2017-05-09 05:04:34,00:00:08,26187.0,2017-05-09 05:54:00,2017-05-09 05:56:51,00:02:51,00:44:04,00:08:13
6,1496,False,4,5,023,BL496,BA140,1.0,1.0,26237.0,2017-05-09 05:57:00,2017-05-09 06:10:24,2017-05-09 05:04:34,00:13:24,26283.0,2017-05-09 06:18:00,2017-05-09 06:30:58,00:12:58,01:05:50,00:20:34
7,1496,False,4,7,304,DL300,BA140,1.0,1.0,26282.0,2017-05-09 06:21:13,2017-05-09 06:32:42,2017-05-09 05:04:34,00:11:29,2654.0,2017-05-09 06:37:00,2017-05-09 06:48:00,00:11:00,01:28:08,00:15:18
8,1869,True,2,1,653,HA270,HA270,1.0,1.0,36174.0,2017-05-09 05:03:02,2017-05-09 05:02:18,2017-05-09 05:03:17,00:00:44,30435.0,2017-05-09 05:20:00,2017-05-09 05:17:17,00:02:43,00:00:59,00:14:59
9,1869,False,2,3,612,HA251,HA270,1.0,1.0,30437.0,2017-05-09 05:26:00,2017-05-09 05:22:34,2017-05-09 05:03:17,00:03:26,28389.0,2017-05-09 05:48:00,2017-05-09 05:44:43,00:03:17,00:19:17,00:22:09


In [191]:
actual_otp_filtered_itineraries.otp_buste_end_timediff.describe()

count                      4985
mean     0 days 00:13:55.480641
std      0 days 00:31:16.657779
min             0 days 00:00:00
25%             0 days 00:01:39
50%             0 days 00:04:13
75%             0 days 00:11:43
max             0 days 09:57:28
Name: otp_buste_end_timediff, dtype: object

In [229]:
actual_otp_filtered_itineraries.boarding_otp_match_start_timediff.describe()

count                      4985
mean     0 days 00:56:55.865997
std      0 days 00:46:16.617581
min             0 days 00:00:01
25%             0 days 00:26:10
50%             0 days 00:54:23
75%             0 days 01:20:41
max             0 days 18:19:17
Name: boarding_otp_match_start_timediff, dtype: object

In [230]:
actual_otp_filtered_itineraries.leg_duration.describe()

count                      4985
mean     0 days 00:11:43.557673
std      0 days 00:31:27.266799
min           -1 days +14:02:36
25%             0 days 00:07:53
50%             0 days 00:15:53
75%             0 days 00:25:29
max             0 days 09:25:28
Name: leg_duration, dtype: object

#### Choosing itinerary

In [246]:
otp_buste_itineraries = actual_otp_filtered_itineraries.groupby(['user_trip_id','itinerary_id']) \
                                .agg({'boarding_otp_match_start_timediff': lambda x: x[0], 'matched_start_time': lambda x: x[0], 'matched_end_time': lambda x: x[-1], 'leg_id': lambda x: len(x)}) \
                                .reset_index() \
                                .rename(index=str,columns={'matched_start_time':'trip_start','matched_end_time':'trip_end','leg_id':'num_transfers','boarding_otp_match_start_timediff':'start_timediff'}) \
                                .assign(trip_duration = lambda x: x['trip_end'] - x['trip_start'])

In [247]:
len(otp_buste_itineraries)

2279

In [248]:
otp_buste_itineraries.head()

,user_trip_id,itinerary_id,trip_start,trip_end,start_timediff,num_transfers,trip_duration
0,1496,2,2017-05-09 05:03:32,2017-05-09 06:29:58,00:01:02,4,01:26:26
1,1496,4,2017-05-09 05:45:57,2017-05-09 06:48:00,00:41:23,4,01:02:03
2,1869,2,2017-05-09 05:02:18,2017-05-09 06:14:06,00:00:59,3,01:11:48
3,1869,4,2017-05-09 05:22:45,2017-05-09 06:29:02,00:19:28,3,01:06:17
4,1869,5,2017-05-09 05:22:45,2017-05-09 06:43:09,00:19:28,4,01:20:24


In [249]:
otp_buste_itineraries = otp_buste_itineraries[otp_buste_itineraries['trip_duration'] > pd.Timedelta('0s')]
otp_buste_itineraries['penalty'] = 2*otp_buste_itineraries['start_timediff'].dt.total_seconds() + otp_buste_itineraries['trip_duration'].dt.total_seconds() + otp_buste_itineraries['num_transfers']*10
otp_buste_itineraries = otp_buste_itineraries.sort_values(['user_trip_id','penalty'], ascending=True)

In [250]:
otp_buste_itineraries.head()

,user_trip_id,itinerary_id,trip_start,trip_end,start_timediff,num_transfers,trip_duration,penalty
0,1496,2,2017-05-09 05:03:32,2017-05-09 06:29:58,00:01:02,4,01:26:26,5350.0
1,1496,4,2017-05-09 05:45:57,2017-05-09 06:48:00,00:41:23,4,01:02:03,8729.0
2,1869,2,2017-05-09 05:02:18,2017-05-09 06:14:06,00:00:59,3,01:11:48,4456.0
3,1869,4,2017-05-09 05:22:45,2017-05-09 06:29:02,00:19:28,3,01:06:17,6343.0
4,1869,5,2017-05-09 05:22:45,2017-05-09 06:43:09,00:19:28,4,01:20:24,7200.0


In [251]:
chosen_itineraries = otp_buste_itineraries.groupby(['user_trip_id']).first().reset_index()

In [252]:
len(chosen_itineraries)

686

In [253]:
chosen_itineraries.head()

,user_trip_id,itinerary_id,trip_start,trip_end,start_timediff,num_transfers,trip_duration,penalty
0,1496,2,2017-05-09 05:03:32,2017-05-09 06:29:58,00:01:02,4,01:26:26,5350.0
1,1869,2,2017-05-09 05:02:18,2017-05-09 06:14:06,00:00:59,3,01:11:48,4456.0
2,1870,6,2017-05-09 16:07:26,2017-05-09 16:43:44,00:11:02,3,00:36:18,3532.0
3,1947,1,2017-05-09 05:00:44,2017-05-09 05:36:59,00:00:24,2,00:36:15,2243.0
4,2028,1,2017-05-09 05:03:54,2017-05-09 05:44:52,00:00:51,3,00:40:58,2590.0


In [254]:
chosen_itineraries.describe()

,user_trip_id,itinerary_id,start_timediff,num_transfers,trip_duration,penalty
count,686.000000,686.000000,686,686.000000,686,686.000000
mean,116932.475219,2.039359,0 days 00:11:26.826530,2.208455,0 days 00:54:50.053935,4685.791545
std,80860.365987,1.902721,0 days 00:34:22.476613,0.824082,0 days 00:43:36.717172,5868.504763
min,1496.000000,1.000000,0 days 00:00:01,1.000000,0 days 00:00:14,204.000000
25%,47572.250000,1.000000,0 days 00:00:20,2.000000,0 days 00:25:25.500000,2630.500000
50%,102281.000000,1.000000,0 days 00:01:01,2.000000,0 days 00:50:14.500000,4182.000000
75%,184387.250000,2.000000,0 days 00:12:43.750000,3.000000,0 days 01:14:11.500000,5453.500000
max,280809.000000,10.000000,0 days 08:33:20,5.000000,0 days 09:18:42,95142.000000


#### Read Bus Trips data

In [166]:
bus_trips = pd.read_csv('/local/tarciso/masters/data/bus_trips/test/enhanced-buste/2017_05_09_bus_trips.csv', parse_dates=['gps_datetime'])

In [148]:
bus_trips.head()

,route,busCode,tripNum,stopPointId,gps_datetime,stop_lat,stop_lon,parent_station
0,654,HA017,2.0,35350,2017-05-09 06:28:49,-25.492665,-49.303977,NaN
1,511,EA172,4.0,31195,2017-05-09 07:47:51,-25.498750,-49.247840,NaN
2,654,HA240,3.0,36094,2017-05-09 06:55:54,-25.501283,-49.320023,NaN
3,175,BC010,5.0,31748,2017-05-09 10:22:28,-25.450968,-49.254072,NaN
4,370,LC016,3.0,3377,2017-05-09 10:42:18,-25.428142,-49.246865,NaN


In [161]:
len(bus_trips)

606996

In [149]:
bus_trips.dtypes

route                     object
busCode                   object
tripNum                  float64
stopPointId                int64
gps_datetime      datetime64[ns]
stop_lat                 float64
stop_lon                 float64
parent_station           float64
dtype: object

#### Find trips whose destination is reacheable from origin by taking the boarding bus

In [164]:
od_trips = bus_trips.merge(trips_origins_clean, left_on=['route','busCode','tripNum'], 
                           right_on=['o_route','o_busCode','o_tripNum'], how='inner') \
                    .sort_values(['cardNum','o_boarding_datetime','gps_datetime']) \
                    .query('gps_datetime > o_boarding_datetime')

In [184]:
len(od_trips)

50987

In [183]:
len(od_trips.drop_duplicates(subset=['cardNum','o_route','o_busCode','o_tripNum','o_boarding_datetime']))

1854

In [166]:
od_trips[['cardNum','o_route','o_busCode','o_tripNum','route','busCode','tripNum','stopPointId','o_boarding_datetime','gps_datetime','next_o_boarding_datetime']] \
        .sort_values(['cardNum','o_boarding_datetime','gps_datetime']).head(50)

,cardNum,o_route,o_busCode,o_tripNum,route,busCode,tripNum,stopPointId,o_boarding_datetime,gps_datetime,next_o_boarding_datetime
85040,331166.0,965,MN606,15.0,965,MN606,15.0,33371,2017-05-09 20:07:29,2017-05-09 20:08:00,2017-05-09 07:53:13
85041,331166.0,965,MN606,15.0,965,MN606,15.0,33306,2017-05-09 20:07:29,2017-05-09 20:09:10,2017-05-09 07:53:13
85042,331166.0,965,MN606,15.0,965,MN606,15.0,33376,2017-05-09 20:07:29,2017-05-09 20:09:34,2017-05-09 07:53:13
85043,331166.0,965,MN606,15.0,965,MN606,15.0,33382,2017-05-09 20:07:29,2017-05-09 20:10:42,2017-05-09 07:53:13
85044,331166.0,965,MN606,15.0,965,MN606,15.0,26346,2017-05-09 20:07:29,2017-05-09 20:11:35,2017-05-09 07:53:13
85045,331166.0,965,MN606,15.0,965,MN606,15.0,33390,2017-05-09 20:07:29,2017-05-09 20:12:13,2017-05-09 07:53:13
85046,331166.0,965,MN606,15.0,965,MN606,15.0,33310,2017-05-09 20:07:29,2017-05-09 20:13:46,2017-05-09 07:53:13
85047,331166.0,965,MN606,15.0,965,MN606,15.0,33450,2017-05-09 20:07:29,2017-05-09 20:14:52,2017-05-09 07:53:13
85048,331166.0,965,MN606,15.0,965,MN606,15.0,26900,2017-05-09 20:07:29,2017-05-09 20:15:37,2017-05-09 07:53:13
85049,331166.0,965,MN606,15.0,965,MN606,15.0,26448,2017-05-09 20:07:29,2017-05-09 20:16:06,2017-05-09 07:53:13


In [167]:
od_trips

,route,busCode,tripNum,stopPointId,gps_datetime,stop_lat,stop_lon,parent_station,cardNum,o_boarding_id,...,next_o_boarding_datetime,next_o_route,next_o_busCode,next_o_tripNum,next_o_stopPointId,next_o_gps_datetime,next_o_stop_lat,next_o_stop_lon,boardings_timediff,dist_between_origins
85040,965,MN606,15.0,33371,2017-05-09 20:08:00,-25.400821,-49.312561,NaN,331166.0,160,...,2017-05-09 07:53:13,011,JB302,2.0,32796,2017-05-09 07:52:52,-25.422572,-49.290002,12:14:16,3.634228
85041,965,MN606,15.0,33306,2017-05-09 20:09:10,-25.404896,-49.313743,NaN,331166.0,160,...,2017-05-09 07:53:13,011,JB302,2.0,32796,2017-05-09 07:52:52,-25.422572,-49.290002,12:14:16,3.634228
85042,965,MN606,15.0,33376,2017-05-09 20:09:34,-25.407791,-49.313072,NaN,331166.0,160,...,2017-05-09 07:53:13,011,JB302,2.0,32796,2017-05-09 07:52:52,-25.422572,-49.290002,12:14:16,3.634228
85043,965,MN606,15.0,33382,2017-05-09 20:10:42,-25.407703,-49.309823,NaN,331166.0,160,...,2017-05-09 07:53:13,011,JB302,2.0,32796,2017-05-09 07:52:52,-25.422572,-49.290002,12:14:16,3.634228
85044,965,MN606,15.0,26346,2017-05-09 20:11:35,-25.429344,-49.271138,NaN,331166.0,160,...,2017-05-09 07:53:13,011,JB302,2.0,32796,2017-05-09 07:52:52,-25.422572,-49.290002,12:14:16,3.634228
85045,965,MN606,15.0,33390,2017-05-09 20:12:13,-25.410597,-49.307468,NaN,331166.0,160,...,2017-05-09 07:53:13,011,JB302,2.0,32796,2017-05-09 07:52:52,-25.422572,-49.290002,12:14:16,3.634228
85046,965,MN606,15.0,33310,2017-05-09 20:13:46,-25.412069,-49.303053,NaN,331166.0,160,...,2017-05-09 07:53:13,011,JB302,2.0,32796,2017-05-09 07:52:52,-25.422572,-49.290002,12:14:16,3.634228
85047,965,MN606,15.0,33450,2017-05-09 20:14:52,-25.413864,-49.298043,NaN,331166.0,160,...,2017-05-09 07:53:13,011,JB302,2.0,32796,2017-05-09 07:52:52,-25.422572,-49.290002,12:14:16,3.634228
85048,965,MN606,15.0,26900,2017-05-09 20:15:37,-25.422483,-49.284328,NaN,331166.0,160,...,2017-05-09 07:53:13,011,JB302,2.0,32796,2017-05-09 07:52:52,-25.422572,-49.290002,12:14:16,3.634228
85049,965,MN606,15.0,26448,2017-05-09 20:16:06,-25.411622,-49.294794,NaN,331166.0,160,...,2017-05-09 07:53:13,011,JB302,2.0,32796,2017-05-09 07:52:52,-25.422572,-49.290002,12:14:16,3.634228


In [168]:
od_trips['dist'] = od_trips.apply(lambda x: dist(x['stop_lat'],x['stop_lon'],x['next_o_stop_lat'],x['next_o_stop_lon']), axis=1)

/local/tarciso/programs/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in arccos
  """


In [169]:
od_trips[['cardNum','o_route','o_busCode','o_tripNum','route','busCode','tripNum','stopPointId','o_boarding_datetime','gps_datetime','next_o_boarding_datetime','dist']].head(30)

,cardNum,o_route,o_busCode,o_tripNum,route,busCode,tripNum,stopPointId,o_boarding_datetime,gps_datetime,next_o_boarding_datetime,dist
85040,331166.0,965,MN606,15.0,965,MN606,15.0,33371,2017-05-09 20:07:29,2017-05-09 20:08:00,2017-05-09 07:53:13,3.314109
85041,331166.0,965,MN606,15.0,965,MN606,15.0,33306,2017-05-09 20:07:29,2017-05-09 20:09:10,2017-05-09 07:53:13,3.090123
85042,331166.0,965,MN606,15.0,965,MN606,15.0,33376,2017-05-09 20:07:29,2017-05-09 20:09:34,2017-05-09 07:53:13,2.840697
85043,331166.0,965,MN606,15.0,965,MN606,15.0,33382,2017-05-09 20:07:29,2017-05-09 20:10:42,2017-05-09 07:53:13,2.587805
85044,331166.0,965,MN606,15.0,965,MN606,15.0,26346,2017-05-09 20:07:29,2017-05-09 20:11:35,2017-05-09 07:53:13,2.038588
85045,331166.0,965,MN606,15.0,965,MN606,15.0,33390,2017-05-09 20:07:29,2017-05-09 20:12:13,2017-05-09 07:53:13,2.202349
85046,331166.0,965,MN606,15.0,965,MN606,15.0,33310,2017-05-09 20:07:29,2017-05-09 20:13:46,2017-05-09 07:53:13,1.755588
85047,331166.0,965,MN606,15.0,965,MN606,15.0,33450,2017-05-09 20:07:29,2017-05-09 20:14:52,2017-05-09 07:53:13,1.260855
85048,331166.0,965,MN606,15.0,965,MN606,15.0,26900,2017-05-09 20:07:29,2017-05-09 20:15:37,2017-05-09 07:53:13,0.569992
85049,331166.0,965,MN606,15.0,965,MN606,15.0,26448,2017-05-09 20:07:29,2017-05-09 20:16:06,2017-05-09 07:53:13,1.309328


In [185]:
od_trips_dest = od_trips.query('dist <= 1.0') \
                        .groupby(['cardNum','o_boarding_datetime','o_route','o_busCode','o_tripNum']) \
                        .first().reset_index() \
                        .sort_values(['cardNum','o_boarding_datetime']) \
                        .assign(trip_duration = lambda x: x['gps_datetime'] - x['o_boarding_datetime'])

In [186]:
od_trips_dest.head()

,cardNum,o_boarding_datetime,o_route,o_busCode,o_tripNum,route,busCode,tripNum,stopPointId,gps_datetime,...,next_o_busCode,next_o_tripNum,next_o_stopPointId,next_o_gps_datetime,next_o_stop_lat,next_o_stop_lon,boardings_timediff,dist_between_origins,dist,trip_duration
0,331166.0,2017-05-09 20:07:29,965,MN606,15.0,965,MN606,15.0,26900,2017-05-09 20:15:37,...,JB302,2.0,32796,2017-05-09 07:52:52,-25.422572,-49.290002,12:14:16,3.634228,0.569992,00:08:08
1,352491.0,2017-05-09 17:32:45,901,MC303,11.0,901,MC303,11.0,33658,2017-05-09 17:48:36,...,MC300,3.0,32369,2017-05-09 07:17:11,-25.402222,-49.327749,10:13:52,5.600896,0.811769,00:15:51
2,384263.0,2017-05-09 11:25:04,916,MA299,3.0,916,MA299,3.0,33341,2017-05-09 12:12:32,...,07001,NaN,14510,NaT,-25.400933,-49.330279,01:20:18,3.093219,0.316732,00:47:28
3,405013.0,2017-05-09 07:27:40,911,MN400,3.0,911,MN400,3.0,34350,2017-05-09 07:33:27,...,07902,NaN,14510,NaT,-25.400933,-49.330279,05:39:03,2.694138,0.910208,00:05:47
4,411687.0,2017-05-09 09:31:13,901,MC303,3.0,901,MC303,3.0,32796,2017-05-09 09:38:08,...,07020,NaN,14432,NaT,-25.422391,-49.291635,07:21:57,2.095410,0.165152,00:06:55


In [187]:
od_trips_dest[['cardNum','o_route','o_busCode','o_tripNum','o_stopPointId','o_boarding_datetime','gps_datetime','next_o_boarding_datetime','dist','trip_duration','boardings_timediff']] \
        .sort_values(['cardNum','o_boarding_datetime'])

,cardNum,o_route,o_busCode,o_tripNum,o_stopPointId,o_boarding_datetime,gps_datetime,next_o_boarding_datetime,dist,trip_duration,boardings_timediff
0,331166.0,965,MN606,15.0,32360,2017-05-09 20:07:29,2017-05-09 20:15:37,2017-05-09 07:53:13,0.569992,00:08:08,12:14:16
1,352491.0,901,MC303,11.0,26722,2017-05-09 17:32:45,2017-05-09 17:48:36,2017-05-09 07:18:53,0.811769,00:15:51,10:13:52
2,384263.0,916,MA299,3.0,33447,2017-05-09 11:25:04,2017-05-09 12:12:32,2017-05-09 12:45:22,0.316732,00:47:28,01:20:18
3,405013.0,911,MN400,3.0,34372,2017-05-09 07:27:40,2017-05-09 07:33:27,2017-05-09 13:06:43,0.910208,00:05:47,05:39:03
4,411687.0,901,MC303,3.0,27256,2017-05-09 09:31:13,2017-05-09 09:38:08,2017-05-09 16:53:10,0.165152,00:06:55,07:21:57
5,445228.0,965,MN606,10.0,33376,2017-05-09 17:10:38,2017-05-09 17:44:13,2017-05-09 06:36:47,0.418205,00:33:35,10:33:51
6,643000.0,967,MN600,3.0,33439,2017-05-09 07:15:07,2017-05-09 07:38:39,2017-05-09 18:32:57,0.668432,00:23:32,11:17:50
7,643000.0,967,MN604,13.0,26345,2017-05-09 18:32:57,2017-05-09 18:54:11,2017-05-09 07:15:07,0.640512,00:21:14,11:17:50
8,652419.0,901,MC491,16.0,26455,2017-05-09 15:06:07,2017-05-09 15:16:34,2017-05-09 13:17:31,0.998556,00:10:27,01:48:36
9,669176.0,967,MN604,2.0,26345,2017-05-09 07:27:21,2017-05-09 07:42:25,2017-05-09 19:13:03,0.999929,00:15:04,11:45:42


In [188]:
len(od_trips_dest)

861

In [189]:
od_trips_dest.trip_duration.describe()

count                       861
mean     0 days 00:14:33.634146
std      0 days 00:16:58.419148
min             0 days 00:00:04
25%             0 days 00:06:45
50%             0 days 00:12:36
75%             0 days 00:19:20
max             0 days 07:06:04
Name: trip_duration, dtype: object

In [181]:
od_trips_dest.dist.describe()

count    861.000000
mean       0.246562
std        0.269727
min        0.000000
25%        0.019221
50%        0.164025
75%        0.381333
max        0.984331
Name: dist, dtype: float64

In [159]:
od_trips_dest.cardNum.value_counts()

3187918.0     2
3250023.0     1
2722488.0     1
1654074.0     1
3788749.0     1
2958180.0     1
771608.0      1
3064671.0     1
1565224.0     1
3498692.0     1
3503554.0     1
384263.0      1
3591477.0     1
2652722.0     1
3484961.0     1
2791181.0     1
2410247.0     1
3804412.0     1
2185602.0     1
3469816.0     1
2054995.0     1
10088111.0    1
3462829.0     1
3636750.0     1
2053596.0     1
3276476.0     1
2704065.0     1
3131077.0     1
3439575.0     1
10077853.0    1
3773414.0     1
2282732.0     1
2346224.0     1
2342388.0     1
3778305.0     1
Name: cardNum, dtype: int64

In [140]:
len(od_trips)

109318

In [27]:
candidate_vehicles = enhanced_buste[(enhanced_buste['parent_station'] == enhanced_buste['stopPointId'].iloc[8]) & 
                               ((pd.to_datetime(enhanced_buste['gps_datetime']) > (pd.to_datetime(enhanced_buste['boarding_datetime'].iloc[8]) - pd.Timedelta(value=5,unit='m'))) & 
                                (pd.to_datetime(enhanced_buste['gps_datetime']) < (pd.to_datetime(enhanced_buste['boarding_datetime'].iloc[8]) + pd.Timedelta(value=20,unit='m'))))]

In [28]:
candidate_vehicles

,cardNum,boarding_datetime,route,busCode,tripNum,gps_datetime,stopPointId,stop_lat,stop_lon,parent_station,shapeId,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,problem,lineName,birthdate,gender
112368,NaN,NaT,203,BD138,1.0,2017-05-01 09:16:17,25732,-25.448323,-49.287777,26105.0,3855.0,4649347.0,-25.448370,-49.287847,5065.989,NO_PROBLEM,NaN,NaN,NaN
114415,NaN,NaT,203,BD147,2.0,2017-05-01 09:26:48,25732,-25.448323,-49.287777,26105.0,3855.0,4649347.0,-25.448370,-49.287847,5065.989,NO_PROBLEM,NaN,NaN,NaN
232947,NaN,NaT,603,HD249,3.0,2017-05-01 09:14:53,25732,-25.448323,-49.287777,26105.0,3156.0,3758012.0,-25.448258,-49.287815,7432.708,NO_PROBLEM,NaN,NaN,NaN
233467,NaN,NaT,603,HD250,2.0,2017-05-01 09:33:05,25732,-25.448323,-49.287777,26105.0,3156.0,3758012.0,-25.448258,-49.287815,7432.708,BETWEEN,NaN,NaN,NaN


In [29]:
def find_candidate_vehicles(row,gps_df,lower_time_thresh = 0, higher_time_thresh = 40):
    return gps_df[(gps_df['parent_station'] == row['stopPointId']) & 
                               ((pd.to_datetime(gps_df['gps_datetime']) > (pd.to_datetime(row['boarding_datetime']) - pd.Timedelta(value=lower_time_thresh,unit='m'))) & 
                                (pd.to_datetime(gps_df['gps_datetime']) < (pd.to_datetime(row['boarding_datetime']) + pd.Timedelta(value=higher_time_thresh,unit='m'))))]

In [30]:
first_boarding = 40
second_boarding = 41

In [31]:
test_data = enhanced_buste.loc[first_boarding:second_boarding].head(1)

In [32]:
test_data

,cardNum,boarding_datetime,route,busCode,tripNum,gps_datetime,stopPointId,stop_lat,stop_lon,parent_station,shapeId,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,problem,lineName,birthdate,gender
40,341036.0,2017-05-01 08:57:07,021,08025,NaN,NaT,41751,-25.441058,-49.3425,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INTERB II ANTI H,09/07/69,F


In [33]:
nstop_lat = enhanced_buste['stop_lat'].loc[second_boarding]
nstop_lon = enhanced_buste['stop_lon'].loc[second_boarding]

In [34]:
nstop_lat, nstop_lon

(-25.4376552204, -49.266631957100003)

In [36]:
dist(-7.213345, -35.907678, -7.219092, -35.877251)

3.4168198120895932

In [37]:
def is_vehicle_suitable(row,gps_df,next_stop_lat,next_stop_lon,higher_time_thresh=2):
    vehicle_trip = gps_df[(gps_df['busCode'] == row['busCode']) & (gps_df['tripNum'] == row['tripNum'])].sort_values('gps_datetime')
    vehicle_suitability = vehicle_trip[(vehicle_trip['gps_datetime'] > row['gps_datetime'])].apply(lambda row: dist(row['stop_lat'],row['stop_lon'],next_stop_lat,next_stop_lon), axis=1) \
                            .reset_index(name='dist')
    #print vehicle_suitability['dist']
    return len(vehicle_suitability[vehicle_suitability['dist'] <= 1.0]) > 0
    

In [38]:
for index, row in test_data.iterrows():
    cand_vehicles = find_candidate_vehicles(row,enhanced_buste)
    suitable_candidates = []
    for cand_index, cand_row in cand_vehicles.iterrows():
        if (is_vehicle_suitable(cand_row,enhanced_buste,nstop_lat, nstop_lon)):
            suitable_candidates.append(cand_row)
    suitable_candidates_df = pd.DataFrame.from_records(suitable_candidates, columns=cand_vehicles.columns)
    print suitable_candidates_df
    chosen_candidate = suitable_candidates_df.loc[suitable_candidates_df['gps_datetime'].idxmin()]
    print chosen_candidate

   cardNum boarding_datetime route busCode  tripNum        gps_datetime  \
0      NaN               NaT   303   DE710      2.0 2017-05-01 09:31:02   
1      NaN               NaT   303   DE713      2.0 2017-05-01 09:17:39   
2      NaN               NaT   303   DE719      2.0 2017-05-01 09:05:31   

   stopPointId   stop_lat   stop_lon  parent_station  shapeId  shapeSequence  \
0        25748 -25.441061 -49.342507         41751.0   3617.0      4399768.0   
1        25748 -25.441061 -49.342507         41751.0   3617.0      4399768.0   
2        25748 -25.441061 -49.342507         41751.0   3617.0      4399768.0   

    shapeLat   shapeLon  distanceTraveledShape     problem  lineName  \
0 -25.441017 -49.342554                456.152  NO_PROBLEM       NaN   
1 -25.441017 -49.342554                456.152  NO_PROBLEM       NaN   
2 -25.441017 -49.342554                456.152  NO_PROBLEM       NaN   

   birthdate  gender  
0        NaN     NaN  
1        NaN     NaN  
2        NaN     NaN